In [11]:
!git clone https://github.com/lxguan1/EECS583_Unroll_ML.git

fatal: destination path 'EECS583_Unroll_ML' already exists and is not an empty directory.


In [14]:
%cd /content/EECS583_Unroll_ML/
!git fetch
!git pull

/content/EECS583_Unroll_ML
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 14 (delta 6), reused 14 (delta 6), pack-reused 0
Unpacking objects: 100% (14/14), done.
From https://github.com/lxguan1/EECS583_Unroll_ML
   36ee999..2cf32ca  main       -> origin/main
Updating 36ee999..2cf32ca
Fast-forward
 data/{unroll_ranks.csv => combined_labels.csv}     |  484 +--
 data/feat_labels_combined.csv                      | 3658 ++++++++++++++++++++
 data/train.csv                                     | 2932 ++++++++++++++++
 .../unroll_labels_5678.csv                         |    0
 data/val.csv                                       |  726 ++++
 model/model.ipynb                                  |   75 +-
 util_scripts/combine_feats_and_labels.py           |   19 +-
 util_scripts/merge_unrolls.py                      |    9 +-
 util_scripts/train_val_split.py                    |   31 +
 util_scripts/

In [43]:
import torch
from torch.utils.data import Dataset, DataLoader    # dataset representation and loading
import torch.nn as nn                     # neural networks
import torch.nn.functional as F           # layers, activations and more
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
import numpy as np
import csv
import pdb

In [6]:
device = None
if torch.cuda.is_available():
    print("Using the GPU")
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')
    print("NOTE: Using CPU")

Using the GPU


In [30]:
class CSVDataset(Dataset): 
    # 9 feats: trip count, num ops, num operands, num mem ops, num fops, num branches, est resmii, frequent path length, depth of loop

    def __init__(self, csv_path):
        csv_reader = csv.reader(open(csv_path, 'r'), delimiter=',')

        # line = [prob_name, f1, ... , f9, rank1, ..., rank8]
        self.csv_lines = []
        for line in csv_reader:
            line = [int(float(item)) for item in line[1:11]]
            self.csv_lines.append(line)
    
    def __len__(self):
        return len(self.csv_lines)
    
    def __getitem__(self, idx):
        line = self.csv_lines[idx]
        X = torch.FloatTensor(line[:-1])
        y = torch.zeros(8)
        y[line[-1] - 1] = 1
        return X, y

In [9]:
class MLP(nn.Module):
    def __init__(self,input_size,hidden_dim_1,hidden_dim_2,num_classes,drop_prob=0.5):
        super(MLP, self).__init__()
        #Put GeneModel architecture here (WITHOUT THE FINAL LAYERS)
        self.hidden_dim_1=hidden_dim_1
        self.hidden_dim_2=hidden_dim_2
        self.num_classes=num_classes
        
        #self.flantten=torch.flatten()
        self.fc_1=nn.Linear(input_size,hidden_dim_1)
        self.fc_2=nn.Linear(hidden_dim_1,hidden_dim_2)
        self.fc_3=nn.Linear(hidden_dim_2,num_classes)

        self.relu = nn.LeakyReLU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):
        #Put GeneModel architecture here (WITHOUT THE FINAL LAYERS)
        x=x.float()
        #x=torch.flatten(x).float()
        x=self.fc_1(x)
        x=self.relu(x)
        x=self.fc_2(x)
        x=self.relu(x)
        x=self.fc_3(x)
        return x
    
    # def train(self, trainloader, optimizer, n_epochs = 30):
    #     train_loss = 0
    #     losses_epochs = []
    #     for epoch in range(n_epochs):
    #         print("Starting epoch: ", epoch)
    #         for X, y in train_batch:
    #             optimizer.zero_grad()

    #             outputs = self(X1, X2)
    #             loss = self.criterion(outputs, y)
    #             loss.backward()
    #             optimizer.step()
    #             train_loss += loss.item()
    #         print("train loss:", train_loss)
    #         losses_epochs.append(train_loss)
    #         train_loss = 0
    #     return losses_epochs

In [44]:
mlp = MLP(input_size=9, hidden_dim_1=8, hidden_dim_2=8, num_classes=8)
train_csv = "/content/EECS583_Unroll_ML/data/train.csv"
val_csv = "/content/EECS583_Unroll_ML/data/val.csv"

lr = 0.001
optimizer = optim.Adam(mlp.parameters(), lr=lr)
epochs = 30
batch_size = 32
class_weights = torch.Tensor([2.4433, 6.319, 8.3295, 7.33, 0.371,0.49, 0.7541, 0.8983])
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Dataloader
train_dataset = CSVDataset(train_csv)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = CSVDataset(val_csv)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [45]:
for i, epoch in enumerate(range(epochs)):
    print(f"EPOCH {i} / {epochs - 1}")
    
    mlp.train()
    train_loss = 0.0
    pdb.set_trace()
    for X, y in train_loader:
        optimizer.zero_grad()
        pred = mlp(X)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print("Training loss =", train_loss / len(train_loader))

    with torch.no_grad():
      mlp.eval()
      val_loss_sum = 0.0
      for X, y in val_loader:
          pred = mlp(X)
          val_loss = mlp.loss_function(pred, y)
          val_loss_sum += val_loss.item()
    print("Validation loss =", val_loss_sum / len(val_loader))
    


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.8/bdb.py", line 334, in set_trace
    sys.settrace(self.trace_dispatch)



EPOCH 0 / 29
> <ipython-input-45-eff780ddd512>(7)<module>()
-> for X, y in train_loader:
(Pdb) n
> <ipython-input-45-eff780ddd512>(8)<module>()
-> optimizer.zero_grad()
(Pdb) X
tensor([[5.0000e+00, 1.1000e+01, 1.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00,
         2.0000e+00, 1.0000e+00, 1.0100e+02],
        [5.0000e+00, 1.1000e+01, 1.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00,
         2.0000e+00, 1.0000e+00, 1.0200e+02],
        [5.7000e+01, 9.6000e+01, 3.4000e+01, 0.0000e+00, 7.0000e+00, 3.4000e+01,
         2.0000e+00, 2.0000e+00, 0.0000e+00],
        [2.6000e+01, 4.3000e+01, 1.5000e+01, 0.0000e+00, 3.0000e+00, 1.5000e+01,
         3.0000e+00, 2.0000e+00, 0.0000e+00],
        [5.2000e+01, 8.8000e+01, 2.4000e+01, 0.0000e+00, 7.0000e+00, 2.4000e+01,
         4.0000e+00, 1.0000e+00, 0.0000e+00],
        [2.3000e+01, 4.4000e+01, 1.2000e+01, 0.0000e+00, 3.0000e+00, 1.2000e+01,
         3.0000e+00, 1.0000e+00, 0.0000e+00],
        [5.0000e+01, 8.3000e+01, 2.7000e+01, 0.0000e+00, 1.00


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.8/bdb.py", line 359, in set_quit
    sys.settrace(None)



BdbQuit: ignored